<a href="https://colab.research.google.com/github/agt7/dev/blob/master/logisticregression_practice_0815.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Submission Format
The submissions should contain the predicted probability of click for each ad impression in the test set using the following format:

id,click

60000000,0.384

63895816,0.5919

759281658,0.1934

895936184,0.9572

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras as K
import tensorflow as tf
import gzip
import random
%matplotlib inline

In [ ]:
random.seed(5566)
num_lines = 40428968 - 1# train.gz file lines counting
sample_size = 2000000
skip = sorted(random.sample(range(1, num_lines), num_lines-sample_size))
skip[0]
# skip = skip[1:] # if contains index=0, the header will be removed

1

In [ ]:
train_file_path = "../input/avazu-ctr-prediction/train.gz"

# skip = sorted(random.sample(range(num_lines), num_lines-sample_size))
train_df = pd.read_csv(
    train_file_path, 
    header=0,
    skiprows=skip)
print(len(train_df))
train_df.head()

2000000


,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,10000679056417042096,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157
1,10010730108771379386,0,14102100,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,1,0,20634,320,50,2374,3,39,-1,23
2,10017325201002992404,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15705,320,50,1722,0,35,-1,79
3,10018897422996679694,0,14102100,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,1,0,20366,320,50,2333,0,39,-1,157
4,10024660782388112342,1,14102100,1005,1,d9750ee7,98572c79,f028772b,ecad2386,7801e8d9,...,1,0,17614,320,50,1993,2,1063,100083,33


In [ ]:
test_df = pd.read_csv("../input/avazu-ctr-prediction/test.gz", header=0)

In [ ]:
submission_df = pd.read_csv("../input/avazu-ctr-prediction/sampleSubmission.gz")

In [ ]:
def to_date_column(df):
    df["dt_hour"] = pd.to_datetime(df["hour"], format="%y%m%d%H")
    df["year"] = df["dt_hour"].dt.year
    df["month"] = df["dt_hour"].dt.month
    df["day"] = df["dt_hour"].dt.day
    df["int_hour"] = df["dt_hour"].dt.hour
    df["is_weekday"] = df["dt_hour"].dt.dayofweek
    df["is_weekend"] = df.apply(lambda x: x["is_weekday"] in [5, 6], axis=1)

In [ ]:
to_date_column(train_df)
train_df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C19,C20,C21,dt_hour,year,month,day,int_hour,is_weekday,is_weekend
0,10000679056417042096,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,35,-1,157,2014-10-21,2014,10,21,0,1,False
1,10010730108771379386,0,14102100,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,39,-1,23,2014-10-21,2014,10,21,0,1,False
2,10017325201002992404,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,35,-1,79,2014-10-21,2014,10,21,0,1,False
3,10018897422996679694,0,14102100,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,39,-1,157,2014-10-21,2014,10,21,0,1,False
4,10024660782388112342,1,14102100,1005,1,d9750ee7,98572c79,f028772b,ecad2386,7801e8d9,...,1063,100083,33,2014-10-21,2014,10,21,0,1,False


In [ ]:
train_df.columns

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'dt_hour', 'year',
       'month', 'day', 'int_hour', 'is_weekday', 'is_weekend'],
      dtype='object')

In [ ]:
train_df.nunique()

id                  2000000
click                     2
hour                    240
C1                        7
banner_pos                7
site_id                3008
site_domain            3482
site_category            24
app_id                 3952
app_domain              246
app_category             26
device_id            278330
device_ip            963716
device_model           5786
device_type               5
device_conn_type          4
C14                    2351
C15                       8
C16                       9
C17                     425
C18                       4
C19                      66
C20                     165
C21                      60
dt_hour                 240
year                      1
month                     1
day                      10
int_hour                 24
is_weekday                7
is_weekend                2
dtype: int64

In [ ]:
label_col = "click"
x_columns = set(list(train_df.columns)) - set(["id", "site_id", "app_id", "hour", "dt_hour", "device_id", "device_ip"] + [label_col])
# x_columns

In [ ]:
x_train = train_df[x_columns]
y_train = train_df[label_col]
# x_train.head()

In [ ]:
x_train.nunique()

device_conn_type       4
is_weekend             2
day                   10
int_hour              24
year                   1
C14                 2351
banner_pos             7
site_domain         3482
device_type            5
device_model        5786
C21                   60
C16                    9
C20                  165
C1                     7
C18                    4
app_category          26
is_weekday             7
C15                    8
site_category         24
C19                   66
app_domain           246
C17                  425
month                  1
dtype: int64

In [ ]:
to_date_column(test_df)
x_test = test_df[x_columns]
# test_df.head()

In [ ]:
del train_df
del test_df

In [ ]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder


x_train_len = len(x_train)
n_df = pd.concat([x_train, x_test])

In [ ]:
d = defaultdict(LabelEncoder)
n_df = n_df.apply(lambda x: d[x.name].fit_transform(x))
n_df.head()

,device_conn_type,is_weekend,day,int_hour,year,C14,banner_pos,site_domain,device_type,device_model,...,C1,C18,app_category,is_weekday,C15,site_category,C19,app_domain,C17,month
0,0,0,0,0,0,538,1,2462,1,2937,...,2,0,0,1,3,0,2,140,106,0
1,0,0,0,0,0,726,1,2144,1,5045,...,2,3,0,1,3,23,4,140,159,0
2,0,0,0,0,0,269,0,4086,1,4093,...,2,0,0,1,3,2,2,140,54,0
3,0,0,0,0,0,691,1,2144,1,2912,...,2,0,0,1,3,23,4,140,149,0
4,0,0,0,0,0,351,1,2586,1,3276,...,2,2,0,1,3,23,52,140,89,0


In [ ]:
# n_df = pd.get_dummies(n_df)

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
scaler.fit(n_df)
n_df = scaler.transform(n_df)
len(n_df)

5577464

In [ ]:
n_x_train = n_df[:x_train_len]
n_x_test = n_df[x_train_len:]

In [ ]:
from sklearn.linear_model import LogisticRegression


assert len(n_x_train) == len(y_train)
clf = LogisticRegression(verbose=True)
clf.fit(n_x_train, y_train)


y_train_star = clf.predict(n_x_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   24.3s finished


In [ ]:
from sklearn.metrics import log_loss


log_loss(y_train, y_train_star)

5.862508288942948

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

accuracy_score(y_train, y_train_star)

0.830263

In [ ]:
confusion_matrix(y_train, y_train_star)

array([[82255,   255],
       [17331,   159]])

In [ ]:
y_test = clf.predict(n_x_test)

In [ ]:
y_test = clf.predict_proba(n_x_test)
print(clf.classes_)
print(y_test.shape)
print(y_test[3:5, ])

y_test = y_test[:, 1]
print(y_test.shape)
print(y_test[3:5])

[0 1]
(4577464, 2)
[[0.87962919 0.12037081]
 [0.89766382 0.10233618]]
(4577464,)
[0.12037081 0.10233618]


In [ ]:
submission_df["click"] = y_test
# y_pred_proba.to_csv('submission.csv', index=False)
submission_df.to_csv("submission.csv", index=False)
submission_df.head()

,id,click
0,10000174058809263569,0.147954
1,10000182526920855428,0.163801
2,10000554139829213984,0.173722
3,10001094637809798845,0.120371
4,10001377041558670745,0.102336
